In [15]:
import pandas as pd
import pyodbc
import json
import requests
import datetime

In [16]:
# 取得 Token , 設置請求參數 (大陸)
url = "https://login.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "473f31e3a6cb9cafaec3ddf36d20092b",
    "client_secret": "ddfed65f5aeebacd0f54eaa464fcb043",
    "redirect_uri": "https://api-scrm.xiaoshouyi.com/",
    "username": "11021041@twkd.com",
    "password": "Marcellin01oiH88cCI"
}

# 發送 post 請求
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

{'access_token': '4ef027cc0f8b667129254da87ff24de3adf74bef9c1cdd80bd1e5544b486cc2b.MjM2OTI2Njk5ODU2MzExMw==0', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2369266998563113', 'login_type': '', 'encryption_key': '', 'client_id': '473f31e3a6cb9cafaec3ddf36d20092b', 'passport_id': '3052172005298442', 'mobile_params': '', 'instance_uri': 'api-scrm.xiaoshouyi.com'}
4ef027cc0f8b667129254da87ff24de3adf74bef9c1cdd80bd1e5544b486cc2b.MjM2OTI2Njk5ODU2MzExMw==0


In [17]:
'''
from 銷售易 account
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
account_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        SELECT 
            id AS accountId,
            accountCode__c AS 公司代号, 
            dimDepart.departName AS 资料区域名称, 
            nation__c AS 国家,
            customItem197__c AS 企业简称, 
            Phone__c AS 固定电话, 
            customItem227__c AS 公司代号_旧,
            customItem266__c AS Inactive, 
            SAP_CompanyID__c AS SAP公司代号, 
            customItem202__c AS 寄送地址,
            customItem198__c.name AS 公司型态,
            customItem297__c AS 勿发型录, 
            customItem298__c AS 勿发短信, 
            customItem299__c AS 勿发礼品, 
            customItem225__c AS 公司公用标签, 
            customItem324__c AS 目标客户类型, 
            parentAccCode1__c AS 关联公司代号,
            customItem335__c AS 经销商签约系列
        FROM 
            account
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    account_df = pd.concat([account_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

account_df = account_df[account_df['公司型态'].str.contains("C|D", na=False)]

account_df['经销商签约系列'] = account_df['经销商签约系列'].astype(str)

account_df = account_df[~account_df['经销商签约系列'].str.contains("全系列", na=False)]

In [18]:
'''
select related_contact
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
contact_related = pd.DataFrame()
 
while True:
    data = {
        "xoql":''' 
            SELECT
            contact__c.contactName 联系人, 
            contactCode__c 联系人代号, 
            customItem11__c 公司代号,
            contactPhone__c 联系人手机号,
            updatedAt 修改日期,
            id 客户联系人关系, 
            customItem38__c 联系人代号_舊,
            customItem17__c 微信号码,
            customItem19__c 职务类别, 
            customItem45__c 免打扰, 
            customItem54__c 联系人资料无效,
            customItem119__c 有效_总,
            customItem25__c 关系状态 
            FROM accContactRelation__c 
            ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    contact_related = pd.concat([contact_related, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

In [19]:
# K_invite (合併相同公司代號的資料)
K_invite = pd.merge(contact_related, account_df, on = '公司代号', how = 'inner')

# 篩選職務類別
K_invite['职务类别']= K_invite['职务类别'].astype(str)
K_invite = K_invite.loc[K_invite['职务类别'].str.contains("负责人|设计总监|设计师|设计助理|设计主管")]

# # 清除關係客戶(聯絡人有主要公司，在此間公司僅為兼職)
# K_invite['关系状态']= K_invite['关系状态'].astype(str)
# K_invite = K_invite[~K_invite['关系状态'].str.contains("挂靠")]

# 有效(总) 為 有效
K_invite['有效_总']= K_invite['有效_总'].astype(str)
K_invite = K_invite[~K_invite['有效_总'].str.contains("无效")]

# 免打擾 不為"勿電訪"
K_invite['免打扰']= K_invite['免打扰'].astype(str)
K_invite = K_invite[~K_invite['免打扰'].str.contains("电访")]

K_invite['修改日期'] = pd.to_datetime(K_invite['修改日期'], unit='ms')
K_invite['修改日期'] = K_invite['修改日期'].dt.strftime('%Y-%m-%d %H:%M:%S')  
K_invite = K_invite.sort_values(by='修改日期', ascending=False) 

### K_invite 清除公司無效
connection_string = "Driver={Devart ODBC Driver for ASE};Database=PRD;Server=192.168.1.211;UID=sapsa;PWD=kd29003039;Port=4901;String Types=Unicode"

connection = pyodbc.connect(connection_string)

sql ='''SELECT SAPSR3.KNA1.KUNNR ,
                  SAPSR3.TKUKT.VTEXT
                      FROM SAPSR3.KNA1
                      LEFT JOIN SAPSR3.TKUKT
                      ON SAPSR3.KNA1.KUKLA=SAPSR3.TKUKT.KUKLA
                      AND SAPSR3.TKUKT.MANDT ='800'
                      AND SAPSR3.TKUKT.SPRAS = 'M'
    '''

df2 = pd.read_sql_query(sql, connection)
connection.close()
df2 = df2.dropna(subset=['VTEXT'])
df2.columns = ["SAP公司代号", "SAP信用管制"]

K_invite = pd.merge(K_invite, df2 ,on ='SAP公司代号',how='left')

K_invite.loc[K_invite['SAP信用管制'] == '管制', '固定电话'] =''

K_invite.loc[K_invite['企业简称'].str.contains
             ("群|项目经理群|已搬|支援|支持|餐休|留守|搬迁|资料不全|关闭|施工方|资料不详|管制|撤走|资料不存|项目不全|资料不足|转行|倒闭|关门|歇业|闭店|关店|已关|删|无效|失效|无望|停业|停用|解散|撤销|撤店|撤离|注销|资料不全|资料暂存|资料不齐|资料重复|木工群|设计师群|设计群|业主群|施工群|施工方群|包工群|业主甲方群|三明客户群|工地群|客户群")
             , 
             '固定电话'] = ''

# 联系人资料无效
K_invite['联系人资料无效'] = K_invite['联系人资料无效'].astype(str)
K_invite['联系人资料无效'] = K_invite['联系人资料无效'].str.strip()
K_invite.loc[K_invite['联系人资料无效']=='True', '固定电话'] = ''

# 公司電話、手機、聯絡人電話與LineID都缺的才刪除
K_invite = K_invite[~((K_invite['固定电话'] == '') & (K_invite['联系人手机号'] == ''))]

# '联系人手机号' 进行筛选，保留手机号码符合特定条件的行
K_invite1 = K_invite[K_invite['联系人手机号'].apply(lambda x: isinstance(x, str) and len(x) == 11 and x != '00000000000')] 

# 添加輔助列，標記是否包含 "上海"
K_invite1['是否上海'] = K_invite1['资料区域名称'].str.contains("上海").astype(int)

# 排序：先按照 "是否上海" 降序排列，再按其他條件（可省略）
K_invite1 = K_invite1.sort_values(by=['是否上海'], ascending=False)

# 刪除重複，保留排序後的最後一個
K_invite1 = K_invite1.drop_duplicates(subset=['联系人手机号'], keep='first')

# 刪除輔助列（如果不需要保留）
K_invite1 = K_invite1.drop(columns=['是否上海'])

# K_invite1 = K_invite1.drop_duplicates(subset=['联系人手机号'], keep='last')

C:\Users\11021041\AppData\Local\Temp\ipykernel_4808\4176285070.py:20: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  K_invite['修改日期'] = pd.to_datetime(K_invite['修改日期'], unit='ms')
C:\Users\11021041\AppData\Local\Temp\ipykernel_4808\4176285070.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(sql, connection)


In [20]:
### 一人公司
K_invite2 = K_invite[~(K_invite['联系人手机号'].isin(K_invite1['联系人手机号'].drop_duplicates(keep=False)))]

# 計算同公司資料數量
value_counts = K_invite2['公司代号'].value_counts()

# 取只有一筆的資料
unique_values = value_counts[value_counts == 1].index.tolist()
K_invite2 = K_invite2[K_invite2['公司代号'].isin(unique_values)]

# 刪除離職員工 = 此公司不存在
contact_related['关系状态']= contact_related['关系状态'].astype(str)
contact_related_resigned = contact_related[contact_related['关系状态'].str.contains('离职')]
K_invite2 = K_invite2[~(K_invite2['联系人代号'].isin(contact_related_resigned['联系人代号'].drop_duplicates(keep=False)))]

In [21]:
# 經營開發客戶分類
K_invite_all = pd.concat([K_invite1,K_invite2]) 

K_invite_all['目标客户类型']= K_invite_all['目标客户类型'].astype(str)
# K_invite_all = K_invite_all[K_invite_all['目标客户类型'] != 'nan']

K_invite_經營 = K_invite_all.loc[K_invite_all['目标客户类型'].str.contains("经营")]
K_invite_開發 = K_invite_all.loc[~K_invite_all['目标客户类型'].str.contains("经营")]

# grouped = K_invite_經營.groupby('资料区域名称').size().reset_index(name='经营') 
# grouped.set_index('资料区域名称', inplace=True)
# grouped1 = K_invite_開發.groupby('资料区域名称').size().reset_index(name='开发')
# grouped1.set_index('资料区域名称', inplace=True)


In [22]:
# url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/xobjects/catalogSendDetail__c/busiType"
# headers = {
#     "Authorization": f"Bearer {ac_token}"
# }
# response2 = requests.get(url_2, headers = headers)
# Task_desc = response2.json()

# Task_desc

In [23]:
# '''
# select 發放明細
# '''

# url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# catalog_detail = pd.DataFrame()
 
# while True:
#     data = {
#         "xoql":''' 
#             SELECT
#             customItem27__c 联系人, 
#             customItem33__c 联系人代号,
#             approvalStatus
#             FROM catalogSendDetail__c 
#             WHERE
#                 entityType = '2808642566788553'

#             ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     catalog_detail = pd.concat([catalog_detail, data], ignore_index=True, sort=False)
    
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']

# catalog_detail['approvalStatus'] = catalog_detail['approvalStatus'].astype(str)
# catalog_detail = catalog_detail[~catalog_detail['approvalStatus'].str.contains('审批通过')]

# catalog_detail

In [24]:
# K_invite_經營開發 = pd.concat([K_invite_經營,K_invite_開發])

# catelog_list = catalog_detail

# # 使用 merge 函数合并两个 DataFrame，根据 "聯繫人代號" 列进行合并
# merged_df = pd.merge(K_invite_經營開發, catelog_list, on="联系人代号", how="inner")

# # 删除 merged_df 中重复的行，保留第一个出现的行
# merged_df = merged_df.drop_duplicates(subset="联系人代号", keep="first")

# # 从 "K_invite_經營開發" 表格中删除与 "catelog_list" 相同 "聯繫人代號" 的行
# K_invite_經營開發 = K_invite_經營開發[~K_invite_經營開發["联系人代号"].isin(merged_df["联系人代号"])]

In [25]:
# K_invite_all = pd.concat([K_invite1,K_invite2]) 

# K_invite_all_count = len(K_invite_all[K_invite_all['资料区域名称'].str.contains('上海')])

# print(K_invite_all_count)

In [26]:
K_invite_經營開發 = pd.concat([K_invite_經營,K_invite_開發])
K_invite_經營開發_count = len(K_invite_經營開發[K_invite_經營開發['资料区域名称'].str.contains('上海')])
print(K_invite_經營開發_count)

12221


In [27]:
import pyodbc
from datetime import datetime

today = datetime.now().date().strftime("%Y-%m-%d")

# 建立連線
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=192.168.1.119;'
                      'DATABASE=bidb;'
                      'UID=kdmis;'
                      'PWD=Kd0123456')

# 建立游標
cursor = conn.cursor()

# SQL 更新語句
sql_update = """
UPDATE [bidb].[dbo].[Daily_country_callnumber]
SET 大陸 = ?
WHERE 日期 = ?
"""

# 執行更新，傳入參數
cursor.execute(sql_update, (K_invite_經營開發_count, today))

# 提交更改
conn.commit()

# 關閉連線
cursor.close()
conn.close()

print("資料更新成功")

資料更新成功


In [28]:
# today = datetime.now().strftime("%Y%m%d")

# K_invite_經營開發.to_excel(f"C:/Users/11021041/銷售易/CN/每日電訪名單/經營開發_all_{today}.xlsx",index = False) 

# print(today)